In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os, sys

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import random
from time import time
import concurrent
from concurrent.futures import ProcessPoolExecutor
from consolidated_runs import run_sergio
from SERGIO.SERGIO.sergio import sergio
import parallel_utils

def new_expected_value_experiment(dataset_id, last_iteration=0, fixed_edge=False, add_edge=True, multiple_edges=False, clean='clean'):
    #print("Running edge finding experiment")
    gt_file = None
    if dataset_id == 1:
        gt_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Interaction_cID_4.txt'
        regs_path = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/Regs_cID_4.txt'
        n_genes = 100
    elif dataset_id == 2:
        gt_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Interaction_cID_5.txt'
        regs_path = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/Regs_cID_5.txt'
        n_genes = 400
    elif dataset_id == 3:
        gt_file = 'SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv'
        target_file = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'
        regs_path = './SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt'
        n_genes = 1200
    reg_df = pd.read_csv(regs_path, header=None)
    master_regs = [int(m) for m in reg_df[0].values]
    true_pearson = pd.DataFrame()
    gt = pd.read_csv(gt_file, header=None)
    imp_dir = os.path.join(os.getcwd(), 'imputations')
    load_dir = os.path.join(imp_dir, f'DS{dataset_id}')
    ranks = []
    means = []

    experiment_dir = os.path.join(os.getcwd(), f'experiments/mean_diffs/DS{dataset_id}')
    if not os.path.exists(experiment_dir):
        os.makedirs(experiment_dir)
    experiment_file = os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment.csv")
    if fixed_edge:
        experiment_file = os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment_fixed_edge.csv")
    if os.path.exists(experiment_file) and last_iteration != 0:
        df = pd.read_csv(experiment_file)
        ranks = df['Rank'].values.tolist()
        means = df['Correlation'].values.tolist()

    #run_sergio(target_file, regs_path, dataset_id, file_extension='')

    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = []
        for iteration in range(last_iteration, 51):      
            file_extension = ''                          
            file_extension = f"_iter{iteration}"
            # Get number of genes to choose a target
            if iteration != 0:
                futures.append(executor.submit(parallel_utils.new_mean_process_iteration, iteration, target_file, regs_path, master_regs, load_dir, add_edge, multiple_edges, imp_dir, dataset_id, file_extension, clean))
        
        iter = 0
        chosen_pairs = []
        chosen_pair_filename = os.path.join(experiment_dir, f"DS{dataset_id}_chosen_pairs.csv")
        expression_data = {}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            expr, clean_data, chosen_targets = future.result()
            expression_data['expr'] = expr
            expression_data['clean'] = clean_data
            # ranks, final_ranks, temp_target, f_ext, ds_iter = future.result()
            # #print(chosen_pair, rank)
            # experiment_file_ranks = experiment_file.replace('.csv', f'_iter{ds_iter}.csv')
            # ranks.to_csv(experiment_file_ranks, index=False)
            # #print(final_ranks)
            # for tup in final_ranks:
            #     chosen_pair = (tup[0], tup[1])
            #     add = tup[2]
            #     rank = ranks.index.get_loc(chosen_pair[1]) + 1
            #     if not add:
            #         rank = n_genes - rank
            #     # print(chosen_pair[0], chosen_pair[1], rank, add)
            #     target_val = tup[4]
            #     target_val = ranks.loc[chosen_pair[1]]['mean']
            #     overall_mean = np.mean(ranks['mean'])
            #     chosen_pairs.append([chosen_pair[0], chosen_pair[1], rank, target_val, overall_mean, add, ds_iter])
        #     if ds_iter % 10 != 0:
        #         #print(temp_target)
        #         os.remove(temp_target)
        #         os.remove(os.path.join(experiment_dir, f"DS{dataset_id}_mean_diff_experiment_iter{ds_iter}.csv"))
        #         os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_clean{f_ext}.npy"))
        #         os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_noisy{f_ext}.npy"))
        #     os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_expr{f_ext}.npy"))
        #     os.remove(os.path.join(imp_dir, f'DS{dataset_id}', f"DS6_clean_counts{f_ext}.npy"))
        #     iter += 1
        # chosen_pair_df = pd.DataFrame(chosen_pairs)
        # chosen_pair_df.to_csv(chosen_pair_filename, index=False)
        n_bins, n_sc = 9, 300
        if dataset_id == 1:
            n_genes = 100
        if dataset_id == 2:
            n_genes = 400
        sim = sergio(
            number_genes=n_genes, 
            number_bins = n_bins, 
            number_sc = n_sc,
            # In paper
            noise_params = 1,
            # In paper
            decays=0.8, 
            sampling_state=15, 
            noise_type='dpd')
        
        outliers = [sim.outlier_effect(expression_data['expr'], outlier_prob = 0.01, mean = 5, scale = 1) for i in range(1)]
        expression_data['outliers'] = outliers
        return expression_data, chosen_targets

In [7]:
expr_data, chosen_targets = new_expected_value_experiment(dataset_id=1, last_iteration=50, fixed_edge=False, add_edge=True, multiple_edges=False, clean='noisy')

  0%|          | 0/1 [00:00<?, ?it/s]/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Instructions for updating:
non-resource variables are not supported in the long term
100%|██████████| 1/1 [00:49<00:00, 49.20s/it]


In [8]:
datasets = expr_data['outliers']

reshaped = [np.concatenate(ds, axis = 1) for ds in datasets]
stacked_data = np.stack(reshaped, axis=2)
consensus_data = np.median(stacked_data, axis=2)

np.corrcoef(consensus_data.flatten(), expr_data['clean'].flatten())[0, 1]
# median_expressions = [np.median(np.abs(norm), axis=2) for norm in datasets]

# stacked_data = np.stack(median_expressions)

# consensus_data = np.median(stacked_data, axis=0)

# variability_scores = np.std(consensus_data, axis=0)

# gene_ranks = np.argsort(-variability_scores)

1.0

In [196]:
nGenes_ = 100
nBins_ = 9
nSC_ = 300
from scipy.stats import norm

def outlier_effect(scData, outlier_prob, mean, scale):
    out_indicator = np.random.binomial(n = 1, p = outlier_prob, size = nGenes_)
    outlierGenesIndx = np.where(out_indicator == 1)[0]
    numOutliers = len(outlierGenesIndx)

    #### generate outlier factors ####
    outFactors = np.random.lognormal(mean = mean, sigma = scale, size = numOutliers)
    ##################################

    scData = np.concatenate(scData, axis = 1)
    for i, gIndx in enumerate(outlierGenesIndx):
        scData[gIndx,:] = scData[gIndx,:] * outFactors[i]

    return np.split(scData, nBins_, axis = 1)

def lib_size_effect(self, scData, mean, scale):
        """
        This functions adjusts the mRNA levels in each cell seperately to mimic
        the library size effect. To adjust mRNA levels, cell-specific factors are sampled
        from a log-normal distribution with given mean and scale.

        scData: the simulated data representing mRNA levels (concentrations);
        np.array (#bins * #genes * #cells)

        mean: mean for log-normal distribution

        var: var for log-normal distribution

        returns libFactors ( np.array(nBin, nCell) )
        returns modified single cell data ( np.array(nBin, nGene, nCell) )
        """

        #TODO make sure that having bins does not intefere with this implementation
        ret_data = []

        libFactors = np.random.lognormal(mean = mean, sigma = scale, size = (nBins_, nSC_))
        for binExprMatrix, binFactors in zip(scData, libFactors):
            normalizFactors = np.sum(binExprMatrix, axis = 0 )
            binFactors = np.true_divide(binFactors, normalizFactors)
            binFactors = binFactors.reshape(1, nSC_)
            binFactors = np.repeat(binFactors, nGenes_, axis = 0)

            ret_data.append(np.multiply(binExprMatrix, binFactors))


        return libFactors, np.array(ret_data)

def outlier_denoise(dataset_list):
    reshaped = [np.concatenate(ds, axis = 1) for ds in dataset_list]
    stacked_data = np.stack(reshaped, axis=2)
    consensus_data = np.median(stacked_data, axis=2)
    return consensus_data

def count_normalization(data):
    normalized_data = []
    for expression_matrix in data:
        totals = np.sum(expression_matrix, axis=0)
        scale_factor = np.median(totals)
        normed = expression_matrix / totals * scale_factor
        normalized_data.append(normed)
    return np.array(normalized_data)

def fit_gaussian(row):
    row_copy = row[row != 0]
    mu, sigma = norm.fit(row_copy)
    return mu, sigma

def dropout_denoise(data, id):
    ids = range(len(data))
    stacked_data = np.stack(data)
    output_array = stacked_data[id]

    # concatenated = np.concatenate(excluded_data, axis=0)
    # concatenated = concatenated.reshape(stacked_data.shape[2], -1)
    # concatenated = pd.DataFrame(concatenated)
    # gaussian_params = concatenated.apply(fit_gaussian, axis=1)
    # print(gaussian_params)
    concatenated = np.concatenate(stacked_data, axis=1)
    concatenated = np.concatenate(concatenated, axis=1)
    concatenated = pd.DataFrame(concatenated)
    #print(concatenated.shape)
    gaussian_params = concatenated.apply(fit_gaussian, axis=1)
    for i in range(stacked_data.shape[1]):
        for j in range(stacked_data.shape[2]):
            gaussian_params_row = gaussian_params.iloc[j + id * stacked_data.shape[2]]
            mu, sigma = gaussian_params_row
            gaussian_sample = norm.rvs(loc=mu, scale=sigma, size=stacked_data.shape[3])
            zeros = np.where(output_array[i, j] == 0)
            output_array[i, j][zeros] = gaussian_sample[zeros]

            # for k in range(stacked_data.shape[3]):
            #     # print(stacked_data[:, i, j, k])
            #     if output_array[i, j, k] == 0:
            #         #value = stacked_data[random_other_id, i, j, k]
            #         #if value == 0:
            #         gaussian_sample = norm.rvs(loc=mu, scale=sigma, size=1)
            #         output_array[i, j, k] = gaussian_sample
                    #else:
                    #    output_array[i, j, k] = value
    return output_array

In [211]:
from SERGIO.SERGIO.sergio import sergio

n_bins, n_sc = 9, 300
dataset_id = 1
if dataset_id == 1:
    n_genes = 100
if dataset_id == 2:
    n_genes = 400
sim = sergio(
    number_genes=n_genes, 
    number_bins = n_bins, 
    number_sc = n_sc,
    # In paper
    noise_params = 1,
    # In paper
    decays=0.8, 
    sampling_state=15, 
    noise_type='dpd')

expr_raw = expr_data['expr']

corr_coeffs = []
diffs= []
for i in range(100):
    outliers = [sim.outlier_effect(expr_raw, outlier_prob = 0.01, mean = 5, scale = 1) for i in range(50)]
    library_data = [sim.lib_size_effect(expr_outlier, mean = 0, scale = 0.1) for expr_outlier in outliers]
    binaries = [sim.dropout_indicator(lib_data[1], shape = 8, percentile = 45) for lib_data in library_data]
    final_expression = [np.multiply(lib_data[1], binary) for lib_data, binary in zip(library_data, binaries)]
    dropout_denoised = [dropout_denoise(final_expression, i) for i in range(len(final_expression))]
    library_denoised = [count_normalization(dropout_denoised[i]) for i in range(len(library_data))]
    #print(library_denoised[0].shape)
    #print(expr_raw.shape)
    for i in range(len(outliers)):
        outlier = outliers[i]
        library = library_data[i][1]
        #print(library.shape)
        outlier = np.array(outlier)
        print("Raw expression to outlier noise correlation")
        print(np.corrcoef(expr_raw.flatten(), outlier.flatten())[0, 1])
        print("Raw expression to library noise correlation")
        other_corr = np.corrcoef(expr_raw.flatten(), library.flatten())[0, 1]
        print(other_corr)
        print("Raw expression to dropout correlation")
        print(np.corrcoef(expr_raw.flatten(), final_expression[i].flatten())[0, 1])
        print("======= Data reconstruction =======")
        print("Dropout noise to library noise correlation")
        initial_corr = np.corrcoef(final_expression[i].flatten(), library.flatten())[0, 1]
        print(initial_corr)
        print("De-noised dropout to library noise correlation")
        after_corr = np.corrcoef(dropout_denoised[i].flatten(), library.flatten())[0, 1]
        print(after_corr)
        print("De-noised dropout to raw expression data correlation")
        after_corr = np.corrcoef(dropout_denoised[i].flatten(), expr_raw.flatten())[0, 1]
        print(after_corr)
        print("De-noised library noise to noisy library noise correlation")
        after_corr = np.corrcoef(library_denoised[i].flatten(), library.flatten())[0, 1]
        print(after_corr)
        print("De-noised library noise to raw expression data correlation")
        after_corr = np.corrcoef(library_denoised[i].flatten(), expr_raw.flatten())[0, 1]
        print(after_corr)

        corr_coeffs.append(after_corr)
        diffs.append(after_corr - other_corr)
        print("\n")
    
    break
    normed = count_normalization(lib_data)
    denoised = outlier_denoise([normed])

    print("Outlier noise to raw expression data correlation")
    print(np.corrcoef(np.array(expr_outlier).flatten(), np.array(expr_raw).flatten())[0, 1])
    out_denoise = outlier_denoise([expr_outlier])
    print("Denoised to outlier noise correlation")
    print(np.corrcoef(out_denoise.flatten(), np.array(expr_outlier).flatten())[0, 1])
    print("Denoised to raw expression data correlation")
    print(np.corrcoef(out_denoise.flatten(), np.array(expr_raw).flatten())[0, 1])

    print("library noise to outlier noise correlation")
    print(np.corrcoef(np.array(lib_data).flatten(), np.array(expr_outlier).flatten())[0, 1])
    print("library noise to raw expression data correlation")
    print(np.corrcoef(np.array(lib_data).flatten(), np.array(expr_raw).flatten())[0, 1])

    print("Normalized to library noise correlation")
    print(np.corrcoef(normed.flatten(), np.array(lib_data).flatten())[0, 1])
    print("Normalized to outlier noise (pre library) correlation")
    print(np.corrcoef(normed.flatten(), np.array(expr_outlier).flatten())[0, 1])
    print("Normalized to raw expression data correlation")
    print(np.corrcoef(normed.flatten(), np.array(expr_raw).flatten())[0, 1])

    corr_coeffs.append(np.corrcoef(denoised.flatten(), np.array(expr_raw).flatten())[0, 1])
    
np.mean(corr_coeffs)

Raw expression to outlier noise correlation
0.053595474706877005
Raw expression to library noise correlation
0.035012445639694696
Raw expression to dropout correlation
0.02329039503889224
======= Data reconstruction =======
Dropout noise to library noise correlation
0.9732915030664939
De-noised dropout to library noise correlation
0.9845208113917494
De-noised dropout to raw expression data correlation
0.02488912406208169
De-noised library noise to noisy library noise correlation
0.9845144697750262
De-noised library noise to raw expression data correlation
0.024964574112818515


Raw expression to outlier noise correlation
1.0
Raw expression to library noise correlation
0.9440830321485416
Raw expression to dropout correlation
0.5246901409494744
======= Data reconstruction =======
Dropout noise to library noise correlation
0.5573480419772753
De-noised dropout to library noise correlation
0.7926784523398619
De-noised dropout to raw expression data correlation
0.760357251867055
De-noised li

0.365881106355427

In [212]:
np.mean(diffs)

-0.08533088711410215

: 